In [173]:
import numpy as np
import pandas as pd
import scipy as sp
import re
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.cross_validation import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.pipeline import FeatureUnion
from sklearn.model_selection import train_test_split

In [174]:
df_legitimate = pd.read_csv("clean_legitimate.txt",names=['url'], header = None, sep = 'asdas2131sad12sa1a11a2sa1',engine='python',encoding = 'utf8').dropna()
df_malicious = pd.read_csv("clean_malicious.txt",names=['url'], header = None, sep = 'asdas2131sad12sa1a11a2sa1',engine='python',encoding = 'utf8').dropna()
X_data = df_legitimate['url'].tolist() + df_malicious['url'].tolist()
y_data = np.zeros(len(df_legitimate)).tolist() + np.ones(len(df_malicious)).tolist()

In [175]:
print(df_malicious.shape)
print(df_legitimate.shape)

(44698, 1)
(1265994, 1)


In [176]:
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.2, random_state=0, shuffle = True)

In [177]:
# marto sityvebis da ricxvebis datoveba
X_train_words = [re.sub("[^A-Za-z0-9]", " ", i) for i in X_train]
X_train_words = [re.sub(' +',' ', i).strip().lower() for i in X_train_words]  
X_test_words = [re.sub("[^A-Za-z0-9]", " ", i) for i in X_test]
X_test_words = [re.sub(' +',' ', i).strip().lower() for i in X_test_words] 

In [178]:
cv = CountVectorizer()
tf_idf = TfidfVectorizer()

x_traincv =cv.fit_transform(X_train_words)
x_testcv=cv.transform(X_test_words)

x_train_tf_idf =tf_idf.fit_transform(X_train_words)
x_test_tf_idf=tf_idf.transform(X_test_words)

y_train= [int(i) for i in y_train]
y_test= [int(i) for i in y_test]

In [179]:
#abrunebs featurebis raodenoba X data Size zomis np.arrays
def get_additional_features(data):
    
    a1  = np.asarray([[int(len(i)<=4) for i in data]])
    a2  = np.asarray([[int(4<len(i)<=12) for i in data]])
    a3  = np.asarray([[int(12<len(i)<=44) for i in data]])
    a4  = np.asarray([[int(44<len(i)) for i in data]])

    a5  = np.asarray([[i.count('/') for i in data]])
    a6  = np.asarray([[i.count('//') for i in data]])
    a7  = np.asarray([[i.count('<') for i in data]])
    a8  = np.asarray([[i.count('>') for i in data]])
    a9  = np.asarray([[i.count('.') for i in data]])
    a10 = np.asarray([[i.count(',') for i in data]])
    a11 = np.asarray([[i.count('..') for i in data]])
    a12 = np.asarray([[i.count('(') for i in data]])
    a13 = np.asarray([[i.count(')') for i in data]])
    a14 = np.asarray([[i.count('[') for i in data]])
    a15 = np.asarray([[i.count(']') for i in data]])
    a16 = np.asarray([[i.count('"') for i in data]])
    a17 = np.asarray([[i.count("'") for i in data]])
    a18 = np.asarray([[i.count('?') for i in data]])
    a19 = np.asarray([[i.count(';') for i in data]])
    a20 = np.asarray([[i.count('-') for i in data]])
    
    a21 = np.asarray([[i.count('https') for i in data]])
    a22 = np.asarray([[(i.count('<')>0 and i.count('>')) for i in data]])
    a23 = np.asarray([[i.count('%') for i in data]])
    a24 = np.asarray([[i.count('+') for i in data]])
    a25 = np.asarray([[i.count('</') for i in data]])
    a26 = np.asarray([[i.count('.exe') for i in data]])
    a27 = np.asarray([[i.count('&') for i in data]])
    a28 = np.asarray([[i.count('=') for i in data]])

    
    return np.concatenate([a1, a2, a3, a4, a5, a6, a7, a8, a9, a10, a11, a12, a13, a14, a15, a16, a17, a18, a19, a20, a21, a22, a23, a24,a25,a26,a27,a28]).T

In [180]:
# featurebis gaertianeba
X_train_final = sp.sparse.hstack([x_traincv, get_additional_features(X_train)])
X_test_final = sp.sparse.hstack([x_testcv, get_additional_features(X_test)])

In [181]:
mnb_model = MultinomialNB()
linear_svc_model = LinearSVC()
logistic_model = LogisticRegression()
random_forest_model = RandomForestClassifier(verbose=3)
svc_model = SVC(verbose=2)

In [74]:
mnb_model.fit(X_train_final, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [75]:
linear_svc_model.fit(X_train_final, y_train) #~2min

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [76]:
logistic_model.fit(X_train_final, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [77]:
random_forest_model.fit(X_train_final, y_train) # ~30min

building tree 1 of 10


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  3.4min remaining:    0.0s


building tree 2 of 10


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  6.4min remaining:    0.0s


building tree 3 of 10
building tree 4 of 10
building tree 5 of 10
building tree 6 of 10
building tree 7 of 10
building tree 8 of 10
building tree 9 of 10
building tree 10 of 10


[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed: 29.9min finished


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=3,
            warm_start=False)

In [ ]:
svc_model.fit(X_train_final, y_train)#too long

[LibSVM]

In [78]:
def confusion(model, X,y):
    predictions=model.predict(X)

    tn, fp, fn, tp = confusion_matrix(predictions, y).ravel()
    total = tn+fp+fn+tp
    actual_no = tn + fp
    actual_yes = tp + fn

    print('tn:' + str(tn), 'fp:' + str(fp))
    print('fn:' + str(fn), 'tp:' + str(tp) +'\n')
    print('Accuracy: ' + str((tp+tn)*100/total))
    print('Misclassification: ' + str((fp+fn)*100/total))
    print('True Positive Rate: ' + str((tp)*100/actual_yes))
    print('False Positive Rat: ' + str((fp)*100/actual_no))
    print('Specificity: ' + str((tn)*100/actual_no))
    print('Precision: ' + str((tp)*100/(tp+fp)))
    print('Prevalence: ' + str(actual_yes*100/total))
    print(tp / (tp+fp))
    print(tn / (tn+fn))

In [79]:
confusion(mnb_model, X_test_final, y_test)

tn:252576 fp:1589
fn:630 tp:7344

Accuracy: 99.15350253109992
Misclassification: 0.8464974689000874
True Positive Rate: 92.09932279909707
False Positive Rat: 0.6251844274388685
Specificity: 99.37481557256113
Precision: 82.21202283667301
Prevalence: 3.0418976192020266
0.8221202283667302
0.9975119073007749


In [80]:
confusion(linear_svc_model, X_test_final, y_test)

tn:253167 fp:170
fn:39 tp:8763

Accuracy: 99.92027130644429
Misclassification: 0.079728693555709
True Positive Rate: 99.55691888207225
False Positive Rat: 0.06710429191156443
Specificity: 99.93289570808844
Precision: 98.09694391581776
Prevalence: 3.357760577403591
0.9809694391581776
0.9998459752138575


In [81]:
confusion(logistic_model, X_test_final, y_test)

tn:253161 fp:266
fn:45 tp:8667

Accuracy: 99.88136065217309
Misclassification: 0.11863934782691625
True Positive Rate: 99.48347107438016
False Positive Rat: 0.10496119198033359
Specificity: 99.89503880801966
Precision: 97.02227695063249
Prevalence: 3.3234276471642907
0.9702227695063249
0.9998222790929124


In [ ]:
confusion(svc_model, X_test_final, y_test)

In [82]:
confusion(random_forest_model, X_test_final, y_test)

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    4.5s remaining:    0.0s


tn:253175 fp:294
fn:31 tp:8639

Accuracy: 99.87601997413586
Misclassification: 0.12398002586414078
True Positive Rate: 99.64244521337947
False Positive Rat: 0.11599051560545866
Specificity: 99.88400948439454
Precision: 96.70883241912011
Prevalence: 3.3074056130526173
0.9670883241912012
0.9998775700417841


[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:   21.4s finished


In [20]:
misclassified = np.where(y_test != logistic_model.predict(X_test_final))

In [83]:
misclassified_indexes = np.where(y_test != mnb_model.predict(X_test_final))
misclassified = [str(y_test[i]) + ' ' + X_test[i] for i in misclassified_indexes[0].tolist()]
misclassified.sort()
misclassified

['0 /%23226%3a+reruns/',
 '0 /%nethood%/',
 '0 /.%2e/.%2e/.%2e/.%2e/winnt/win.ini',
 '0 //%2e%2e%5c%2e%2e%5c%2e%2e%5c%2e%2e%5c%2e%2e%5c%2e%2e%5c%2e%2e%5c%2e%2e%5cwindows/win.ini',
 '0 //%uff0e%uff0e/%uff0e%uff0e/%uff0e%uff0e/%uff0e%uff0e/%uff0e%uff0e/%uff0e%uff0e/%uff0e%uff0e/%uff0e%uff0e/windows/win.ini',
 '0 //..%5c..%5c..%5c..%5c..%5c..%5c..%5c..%5cwindows/win.ini',
 '0 //................windows/win.ini',
 '0 //q32152879&#x26q1741&#x22q1741&#x3cq1741&#x3eq32152879',
 '0 //q59551631&#38q2217&#34q2217&#60q2217&#62q59551631',
 '0 /?pattern=/etc/*&sort=name',
 '0 /\\xc0.\\xc0./\\xc0.\\xc0./\\xc0.\\xc0./\\xc0.\\xc0./\\xc0.\\xc0./winnt/win.ini',
 '0 /admin/config.php?handler=file&module=dashboard&file=dashboard.css',
 '0 /admin/config.php?type=setup&display=extensions&extdisplay=6004',
 '0 /admin/config.php?type=setup&display=extensions&extdisplay=6008',
 '0 /admin/config.php?type=setup&display=extensions&extdisplay=`ls`&action=del',
 '0 /admin/login.php?action=insert&password=test&userna

['1 /basexssl/',
 '1 /aladdinghostscript/',
 '1 /subscriptors/',
 '0 /forum.php?forum=-1 union select password,password,null,null from test_mysql_injection where uid=1/*',
 '1 /job descriptions/',
 '0 /document_imaging/',
 '1 /prescriptives-coupons/',
 '1 /0x2e0x2e0x5c0x2e0x2e0x5c0x2e0x2e0x5c0x2e0x2e0x5c{FILE}',
 '1 /en-us/manager/search/data/inputs/script',
 '1 /javascript/document.jsf',
 '1 /javascript/document.sfish',
 '1 //_mmserverscripts/mmhttpdb.php',
 '1 /scriptextensions/',
 '1 /%2e%2e%2f%2e%2e%2f{FILE}',
 '0 /phpmyadmin//..%5c..%5c..%5c..%5c..%5c..%5c..%5c..%5cwindows/win.ini',
 '1 /scriptpad/',
 '1 /en-us/manager/permissions/launcher/data/ui/nav/default?uri=/servicesns/nobody/xss/data/ui/nav/default',
 '0 /javascript/scripts.cpp',
 '1 /jobdescriptionandsalary/',
 '1 exec xp',
 '1 \\u003C',
 '1 /en-us/manager/search/data/inputs/script/_new?action=edit&ns=search',
 '1 /javascript/document.vbs',
 '1 /virus_descriptions/',
 '0 /document_collaboration/',
 '1 /course_descriptions/

In [84]:
misclassified_indexes = np.where(y_test != linear_svc_model.predict(X_test_final))
misclassified = [str(y_test[i]) + ' ' + X_test[i] for i in misclassified_indexes[0].tolist()]
misclassified.sort()
misclassified

['0 //%2e%2e%5c%2e%2e%5c%2e%2e%5c%2e%2e%5c%2e%2e%5c%2e%2e%5c%2e%2e%5c%2e%2e%5cwindows/win.ini',
 '0 //%uff0e%uff0e/%uff0e%uff0e/%uff0e%uff0e/%uff0e%uff0e/%uff0e%uff0e/%uff0e%uff0e/%uff0e%uff0e/%uff0e%uff0e/windows/win.ini',
 '0 //..%5c..%5c..%5c..%5c..%5c..%5c..%5c..%5cwindows/win.ini',
 '0 //................windows/win.ini',
 '0 /260036(8)_order/',
 '0 /admin/config.php?handler=file&module=dashboard&file=dashboard.css',
 '0 /amateurs having a nice threesome/',
 '0 /blitz-document/',
 '0 /choicepoints_or/',
 '0 /crumb_or/',
 '0 /document-library/',
 '0 /document-viewer/',
 '0 /document_001/',
 '0 /document_collaboration/',
 '0 /document_integration/',
 '0 /document_protection/',
 '0 /document_types/',
 '0 /document_verifi/',
 '0 /en-us/api/search/jobs?s=1331892428.4&s=1331892428.5&s=1331892428.7&s=1331892428.8&s=1331892428.9&s=1331892429.15&s=1331892429.16&s=1331892429.17',
 '0 /ewebquiz.asp?quizid=1 or 1=1',
 '0 /examples/jsp/jsp2/el/.|./.|./.|./.|./.|./.|./.|./.|./.|./.|./.|./.|./win

In [85]:
misclassified_indexes = np.where(y_test != logistic_model.predict(X_test_final))
misclassified = [str(y_test[i]) + ' ' + X_test[i] for i in misclassified_indexes[0].tolist()]
misclassified.sort()
misclassified

['0 //%2e%2e%5c%2e%2e%5c%2e%2e%5c%2e%2e%5c%2e%2e%5c%2e%2e%5c%2e%2e%5c%2e%2e%5cwindows/win.ini',
 '0 //%uff0e%uff0e/%uff0e%uff0e/%uff0e%uff0e/%uff0e%uff0e/%uff0e%uff0e/%uff0e%uff0e/%uff0e%uff0e/%uff0e%uff0e/windows/win.ini',
 '0 //..%5c..%5c..%5c..%5c..%5c..%5c..%5c..%5cwindows/win.ini',
 '0 //................windows/win.ini',
 '0 /admin/config.php?handler=file&module=dashboard&file=dashboard.css',
 '0 /blitz-document/',
 '0 /cgi-bin/where.pl?sd=ls /etc',
 '0 /document-library/',
 '0 /document-viewer/',
 '0 /document_001/',
 '0 /document_collaboration/',
 '0 /document_imaging/',
 '0 /document_integration/',
 '0 /document_protection/',
 '0 /document_types/',
 '0 /document_verifi/',
 '0 /en-us/api/search/jobs?s=1331892428.4&s=1331892428.5&s=1331892428.7&s=1331892428.8&s=1331892428.9&s=1331892429.15&s=1331892429.16&s=1331892429.17',
 '0 /ewebquiz.asp?quizid=1 or 1=1',
 '0 /forged_document/',
 '0 /forum.php?forum=-1 union select password,password,null,null from test_mysql_injection where ui

In [86]:
misclassified_indexes = np.where(y_test != random_forest_model.predict(X_test_final))
misclassified = [str(y_test[i]) + ' ' + X_test[i] for i in misclassified_indexes[0].tolist()]
misclassified.sort()
misclassified

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    4.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:   21.3s finished


['0 * secure-',
 '0 /.%2e/.%2e/.%2e/.%2e/winnt/win.ini',
 '0 //................windows/win.ini',
 '0 /boot.ini',
 '0 /cgi-bin/hw3.php?daysonly=0).system(id).(',
 '0 /classes/main_class.php?default_path=413415588095\\x00',
 '0 /core/admin/edit.php?c=ok&do=edit&p=admin&absoluteurl@rfiurl',
 '0 /core/admin/upload.php?c=ok&do=upload&p=admin&absoluteurl@rfiurl',
 '0 /en-us/account/hw3.php?daysonly=0).system(id).(',
 '0 /examples/servlets/servlet/.|./.|./.|./.|./.|./.|./.|./.|./.|./.|./.|./.|./winnt/win.ini',
 '0 /help.php?q=&rm q96999374 #',
 '0 /help.php?q=\\x0auname\\x09#',
 '0 /http%3a%2f%2fwww/',
 '0 /inc/articles.inc.php?globals[cheminmodules]=@rfiurl',
 '0 /includes/ordersuccess.inc.php?cart_order_id=1&glob=1&glob[rootdir]=@rfiurl',
 '0 /index.php?action=information&informationid=1 and generate_error=error',
 '0 /interface/main/calendar/index.php?module=postcalendar&viewtype=day&func=view&pc_username=admin&framewidth=934',
 '0 /javascript/scripts.pl',
 '0 /javascript/scripts.sh',
 '0 

In [172]:
#saving models
import pickle
pickle.dump(mnb_model, open('Trained Models\mnb_model', 'wb'))
pickle.dump(linear_svc_model, open('Trained Models\linear_svc_model', 'wb'))
pickle.dump(logistic_model, open('Trained Models\logistic_model', 'wb'))
pickle.dump(random_forest_model, open('Trained Models\\random_forest_model', 'wb'))
with open("Trained Models\Saved_CountVectorizer.pkl", 'wb') as handle:
                    pickle.dump(cv, handle)
        
#loaded_model = pickle.load(open(filename, 'rb'))

In [135]:
get_additional_features(['asdas/safdsasdas'])

array([[0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0]])

In [169]:
def string_to_model_input(text):
    
    text_words = re.sub("[^A-Za-z0-9]", " ", text)
    sparse_text_words = loadedcv.transform([re.sub("[^A-Za-z0-9]", " ", text_words)])
    additional_data = get_additional_features([text])
    features = sp.sparse.hstack([sparse_text_words,additional_data])
    return features

In [170]:
def test_model(model, url):
    predict = int(model.predict(string_to_model_input(url)))
    return 'Malicious' if predict == 1 else 'Legitimate'

In [171]:
test_model(linear_svc_model, 'password??????%%%%%%%%%%%')

'Malicious'